# Kryptografia wizualna

In [1]:
import time
import os.path
import random
import numpy
from matplotlib import pyplot
from ipywidgets import interact, Dropdown, Button, Text, Box

In [2]:
def convert_to_black_and_white(image: numpy.ndarray) -> numpy.ndarray:
    if len(image.shape) == 3:
        if image.shape[2] == 4:
            image = image[:, :, :3]

        if image.shape[2] == 3:
            image = image.sum(axis=2) / 3

    if issubclass(image.dtype.type, numpy.floating):
        image = image * 255

    image_bw = numpy.zeros_like(image, dtype=bool)
    image_bw[image > 127] = True
    return image_bw

In [3]:
class Encryptor:
    def __init__(self, text1: Text, text2: Text, button: Button):
        self.image1: numpy.ndarray = None
        self.image2: numpy.ndarray = None
        self.text1 = text1
        self.text2 = text2
        self.button = button

    def encrypt(self, filename: str):
        if filename is None:
            return
        
        self.button.enabled = False
        self.button.description = "Computing"

        _, (ax1, ax2, ax3) = pyplot.subplots(1, 3, sharey=True, figsize=[10.0, 4.8], layout="tight")
        
        image = pyplot.imread(filename)
        image = convert_to_black_and_white(image)

        ax1.imshow(image, cmap="gray")

        self.image1 = numpy.zeros((image.shape[0], image.shape[1] * 2), dtype=bool)
        self.image1[:, ::2] = numpy.random.randint(0, high=2, size=image.shape, dtype=bool)
        self.image1[:, 1::2] = numpy.logical_not(self.image1[:, ::2])
        self.image2 = numpy.copy(self.image1)
        self.image2[:, ::2][image] = numpy.logical_not(self.image2[:, ::2][image])
        self.image2[:, 1::2] = numpy.logical_not(self.image2[:, ::2])
        
        ax2.imshow(self.image1, cmap="gray")
        ax3.imshow(self.image2, cmap="gray")
        pyplot.show()

        self.button.description = "Save result"
        self.button.enabled=True
        
        
    def save(self, *args):
        self.button.disabled = False
        
        if self.image1 is None or self.image2 is None:
            self.button.description = "Nothing to save :("
        elif self.text1.value is None or self.text2.value is None:
            self.button.description = "Specify save file names"
        else:
            pyplot.imsave(self.text1.value, self.image1, cmap="gray")
            pyplot.imsave(self.text2.value, self.image2, cmap="gray")
            self.button.description = "Saved!"

        time.sleep(2)
        self.button.description = "Save result"
        self.button.enabled = True
        

In [4]:
class Decryptor:
    def __init__(self, text: Text, button: Button):
        self.image: numpy.ndarray = None
        self.text = text
        self.button = button

    def decrypt(self, filename1: str, filename2: str):
        if filename1 is None or filename2 is None:
            return

        self.button.enabled = False
        self.button.description = "Computing"

        _, (ax1, ax2, ax3) = pyplot.subplots(1, 3, sharey=True, figsize=[10.0, 4.8], layout="tight")

        image1 = pyplot.imread(filename1)
        image2 = pyplot.imread(filename2)

        if image1.shape != image2.shape:
            raise ValueError("Images differ in size")

        image1 = convert_to_black_and_white(image1)
        image2 = convert_to_black_and_white(image2)

        ax1.imshow(image1, cmap="gray")
        ax2.imshow(image2, cmap="gray")

        self.image = numpy.logical_or(image1, image2)
        
        ax3.imshow(self.image, cmap="gray")
        pyplot.show()

        self.button.description = "Save result"
        self.button.enabled=True
        
        
    def save(self, *args):
        self.button.disabled = False
        
        if self.image is None:
            self.button.description = "Nothing to save :("
        elif self.text.value is None:
            self.button.description = "Specify save file name"
        else:
            pyplot.imsave(self.text.value, self.image, cmap="gray")
            self.button.description = "Saved!"

        time.sleep(2)
        self.button.description = "Save result"
        self.button.enabled = True
        

## Encryption

In [5]:
text1 = Text(description="Output1:")
text2 = Text(description="Output2:")
text1.value="first.png"
text2.value="second.png"
button = Button(description="Save result", enabled=False)
encryptor = Encryptor(text1, text2, button)

filenames = next(os.walk("."))[2]
interact(encryptor.encrypt, filename=Dropdown(options=filenames, value=None, description="Input:"))

button.on_click(encryptor.save)
display(Box([text1, text2, button]));

interactive(children=(Dropdown(description='Input:', options=('first.png', 'original.png', 'reproduction.png',…

Box(children=(Text(value='first.png', description='Output1:'), Text(value='second.png', description='Output2:'…

## Decryption

In [6]:
text = Text(description="Output:")
text.value="reproduction.png"
button = Button(description="Save result", enabled=False)
decryptor = Decryptor(text, button)

filenames = next(os.walk("."))[2]
interact(
    decryptor.decrypt,
    filename1=Dropdown(options=filenames, value=None, description="Input1:"),
    filename2=Dropdown(options=filenames, value=None, description="Input2:")
)

button.on_click(decryptor.save)
display(Box([text, button]));

interactive(children=(Dropdown(description='Input1:', options=('first.png', 'original.png', 'reproduction.png'…

Box(children=(Text(value='reproduction.png', description='Output:'), Button(description='Save result', style=B…